In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data = pd.read_csv("spanish-cities-energy-consumption/weather_features.csv")
data.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>dt_iso</th>
      <th>city_name</th>
      <th>temp</th>
      <th>temp_min</th>
      <th>temp_max</th>
      <th>pressure</th>
      <th>humidity</th>
      <th>wind_speed</th>
      <th>wind_deg</th>
      <th>rain_1h</th>
      <th>rain_3h</th>
      <th>snow_3h</th>
      <th>clouds_all</th>
      <th>weather_id</th>
      <th>weather_main</th>
      <th>weather_description</th>
      <th>weather_icon</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2015-01-01 00:00:00+01:00</td>
      <td>Valencia</td>
      <td>270.475</td>
      <td>270.475</td>
      <td>270.475</td>
      <td>1001</td>
      <td>77</td>
      <td>1</td>
      <td>62</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0</td>
      <td>800</td>
      <td>clear</td>
      <td>sky is clear</td>
      <td>01n</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2015-01-01 01:00:00+01:00</td>
      <td>Valencia</td>
      <td>270.475</td>
      <td>270.475</td>
      <td>270.475</td>
      <td>1001</td>
      <td>77</td>
      <td>1</td>
      <td>62</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0</td>
      <td>800</td>
      <td>clear</td>
      <td>sky is clear</td>
      <td>01n</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2015-01-01 02:00:00+01:00</td>
      <td>Valencia</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>1002</td>
      <td>78</td>
      <td>0</td>
      <td>23</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0</td>
      <td>800</td>
      <td>clear</td>
      <td>sky is clear</td>
      <td>01n</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2015-01-01 03:00:00+01:00</td>
      <td>Valencia</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>1002</td>
      <td>78</td>
      <td>0</td>
      <td>23</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0</td>
      <td>800</td>
      <td>clear</td>
      <td>sky is clear</td>
      <td>01n</td>
    </tr>
    <tr>
      <th>4</th>
      <td>2015-01-01 04:00:00+01:00</td>
      <td>Valencia</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>269.686</td>
      <td>1002</td>
      <td>78</td>
      <td>0</td>
      <td>23</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0</td>
      <td>800</td>
      <td>clear</td>
      <td>sky is clear</td>
      <td>01n</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
data.describe()

In [ ]:
for col, dtype in data.dtypes.items():
    print(f"Column '{col}' has data type: {dtype}")

In [ ]:
data.isna().sum()

In [ ]:
print(data.isnull().values.any())

In [ ]:
for col in z_scores.columns:
    # Plot Histogram of Z-scores
    plt.figure(figsize=(12, 5))

    # Histogram plot
    plt.subplot(1, 2, 1)
    plt.hist(z_scores[col], bins=100, edgecolor='k', alpha=0.7)
    plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (+{threshold})')
    plt.axvline(-threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (-{threshold})')
    plt.title(f'Histogram of Z-scores {col}')
    plt.xlabel('Z-score')
    plt.ylabel('Frequency')
    plt.yscale('log')
    plt.legend()

    # Scatter Plot with Z-scores
    plt.subplot(1, 2, 2)
    plt.scatter(range(len(z_scores[col])), z_scores[col], c=['red' if outlier else 'blue' for outlier in outliers[col]], alpha=0.7)
    plt.axhline(threshold, color='r', linestyle='dashed', linewidth=1.5)
    plt.axhline(-threshold, color='r', linestyle='dashed', linewidth=1.5)
    plt.title(f'Scatter Plot of Z-scores {col}')
    plt.xlabel('Index')
    plt.ylabel('Z-score')
    plt.grid(True)

    # Highlight outliers
    for i, (score, is_outlier) in enumerate(zip(z_scores[col], outliers)):
        if is_outlier:
            plt.text(i, score, f'{score:.2f}', fontsize=9, color='red', ha='left', va='bottom')

    plt.tight_layout()
    plt.show()